In [1]:
# 📊 STEP 1: Load trade journal
import sys
sys.path.append("../") 

import json
from pathlib import Path

journal_path = Path("data/journal.json")

if journal_path.exists():
    with open(journal_path, "r") as f:
        journal = json.load(f)
    print(f"✅ Loaded {len(journal)} journal entries.")
else:
    print("❌ No journal found. Run previous steps to generate trade logs.")
    journal = []


✅ Loaded 14 journal entries.


In [3]:
# 📘 STEP 2: Prepare trade history for GPT (with safeguards)
explanations = [entry["explanation"] for entry in journal if "explanation" in entry]
reflections = [
    entry["reflections"]
    for entry in journal
    if "reflections" in entry
    and isinstance(entry["reflections"], dict)
    and all(k in entry["reflections"] for k in ("mindset", "rules_followed", "lesson"))
]

trade_history_summary = "\n\n".join(explanations[:10])
mindset_summary = "\n".join([
    f"- 🧠 {r['mindset']}, 📏 Rules: {r['rules_followed']}, 📚 Lesson: {r['lesson']}"
    for r in reflections
])

print("📖 Preview GPT Input (trades + mindset):\n")
print(trade_history_summary[:1000])  # preview



📖 Preview GPT Input (trades + mindset):

Sure, let's break down the calendar call spread strategy in simple terms:

**What is a Calendar Call Spread?**
A calendar call spread involves buying and selling call options with the same strike price but different expiration dates.

**Trade Details:**
- Ticker: SPY is the ETF representing the S&P 500 index.
- Strategy: Calendar call spread means buying a longer-term call option and selling a shorter-term call option.
- Strike Price: Both the short and long call options have a strike price of 520.0.
- Expiration Dates: The short call expires on April 7, 2025, and the long call expires on April 8, 2025.
- Net Debit: The trade costs 1.62 per share to enter.

**Profit Potential:**
- The goal of a calendar call spread is to profit from the difference in time decay between the short and long options.
- If the stock price stays near the strike price (520.0) until the short option expires, the trade can potentially benefit from the accelerated time de

In [5]:
# 🤖 STEP 3: Build a strategy using GPT
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

prompt = f"""
You are an elite options strategist and performance coach.

Analyze the following trading history and mindset reflections to identify patterns of success.
Then, suggest **3 new option strategies** that align with the trader’s strengths.

--- TRADE EXPLANATIONS ---
{trade_history_summary}

--- MINDSET REFLECTIONS ---
{mindset_summary}

Return your response in this format:

1. 📈 Strategy Name:
   - Description:
   - Why this fits the trader:
   - Best conditions for this strategy:
"""

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You're a trading mentor and strategy optimizer."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7
)

strategy_output = response.choices[0].message.content.strip()


In [6]:
# 💾 STEP 4: Output the GPT-generated strategies
print("📌 GPT Strategy Recommendations:\n")
print(strategy_output)

# Optional: Save it
output_path = Path("data/strategies_generated.json")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, "w") as f:
    json.dump({"strategies": strategy_output}, f, indent=2)

print(f"\n✅ Strategies saved to {output_path}")


📌 GPT Strategy Recommendations:

**Pattern of Success**

The trader's historical performance and mindset reflections show a consistent use of the calendar call spread strategy, which indicates a preference for strategies that profit from time decay and stable market conditions. The repeated use of improvisation as a rule indicates an adaptive mindset, and the consistent learning mindset shows a willingness to continually improve and adjust their strategy. This suggests the trader might do well with strategies that allow for adjustments and can benefit from changing market conditions.

Based on this, here are three new option strategies that align with the trader’s strengths:

1. 📈 Strategy Name: Iron Condor
   - Description: An Iron Condor involves selling a call spread and a put spread on the same underlying asset with the same expiration date. The goal is to profit from a lack of movement in the underlying asset.
   - Why this fits the trader: Similar to a calendar spread, an Iron Co